In [ ]:
!pwd

In [ ]:
from dotenv import load_dotenv
import os

from langchain_openai import AzureChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

load_dotenv(override=True)

model = AzureChatOpenAI(
    azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version = os.getenv('AZURE_OPENAI_API_VERSION'),
    api_key = os.getenv('AZURE_OPENAI_API_KEY'),
    deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
    # callbacks=[human_intervention_handler]
)

## server parameters
server_params={
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/Users/aronhorvath/Documents/projects/interviews/ibusiness/mcp_tutorial/servers/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            "command": "python",
            # Make sure to update to the full absolute path to your weather_server.py file
            "args": ["/Users/aronhorvath/Documents/projects/interviews/ibusiness/mcp_tutorial/servers/weather_server.py"],
            "transport": "stdio",
        }
    }

client = MultiServerMCPClient(server_params)
tools = await client.get_tools()
agent = create_react_agent(model, tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})
print(math_response.messages[-1])

In [ ]:
math_response['messages'][-1].content

In [ ]:
weather_response['messages'][-1].content